# Run the other two notebooks first before running this!

This notebook assumes that you've 1) downloaded the initial dataset, and 2) ran the Branded Food Initial Data Cleaning notebook.

In [1]:
import os
import psycopg2
import pandas as pd
from config import config # This is a file to get params as a dict for the db login, make your own!

In [2]:
reduced_food = pd.read_csv(os.path.join('cleaned', 'branded_food_reduced.csv')).drop(columns='Unnamed: 0')
reduced_food.head()

C:\Users\Alexander\AppData\Local\Temp\ipykernel_15336\388085391.py:1: DtypeWarning: Columns (2,5,9,10,31) have mixed types. Specify dtype option on import or set low_memory=False.
  reduced_food = pd.read_csv(os.path.join('cleaned', 'branded_food_reduced.csv')).drop(columns='Unnamed: 0')


,fdc_id,gtin_upc,serving_size,serving_size_unit,household_serving_fulltext,package_weight,modified_date,available_date,preparation_state_code,short_description,...,insig_total_sugars,insig_calories_from_fat,insig_sugars,insig_fiber,insig_other_nutrients,household_serving_amount,household_serving_unit,category_id,brand_owner_id,brand_name_id
0,1105904,27000612323,15.00,ml,NaN,NaN,2020-10-02,2020-11-13,NaN,NaN,...,0,0,0,0,0,NaN,NaN,164,0.0,NaN
1,605334,27000690260,15.00,ml,1 tbsp (15ml),NaN,2019-03-25,2019-12-06,NaN,NaN,...,0,0,0,0,0,1.00,tbsp,164,1.0,NaN
2,606302,64144555550,0.25,g,1/4 Second Spray (0.25g),NaN,2019-03-17,2019-12-06,NaN,NaN,...,0,0,0,0,0,0.25,second spray,164,1.0,NaN
3,606392,64144033164,0.25,g,1/4 second spray (0.25g),NaN,2019-03-25,2019-12-06,NaN,NaN,...,0,0,0,0,0,0.25,second spray,164,1.0,NaN
4,607224,64144048502,0.25,g,1/4 Second Spray (.25g),NaN,2018-10-21,2019-12-06,NaN,NaN,...,0,0,0,0,0,0.25,second spray,164,1.0,NaN


fdc_id                          int64
gtin_upc                       object
serving_size                  float64
serving_size_unit              object
household_serving_fulltext     object
package_weight                 object
modified_date                  object
available_date                 object
preparation_state_code         object
short_description              object
GDSN                             bool
LI                               bool
NZGDSN                           bool
insig_iron                      int64
insig_calcium                   int64
insig_cholesterol               int64
insig_dietary_fiber             int64
insig_trans_fat                 int64
insig_satured_fat               int64
insig_vitamin_d                 int64
insig_potassium                 int64
insig_vitamin_a                 int64
insig_vitamin_c                 int64
insig_added_sugars              int64
insig_total_sugars              int64
insig_calories_from_fat         int64
insig_sugars

In [27]:
def connect(command_text):
    """Connects to the database and returns an sql cursor."""
    print('Attempting to connect to postgreSQL database...')
    connection = None
    crsr = None
    try:
        # Connect to db
        connection = psycopg2.connect(**config())
        print('Connected')
        # Get cursor and fetch version
        crsr = connection.cursor()
        crsr.execute('SELECT version()')
        db_version = crsr.fetchone() # Fetch first row?
        print('postgreSQL db version: {0}'.format(db_version))
    except(Exception, psycopg2.DatabaseError) as error:
        print(error)
        
    # Exit incase of error
    if crsr is None:
        return None, None
    
    try:
        crsr.execute(command_text)
    except(psycopg2.InternalError) as error:
        print(error)
        connection.rollback()
    

In [4]:
connection, cursor = connect()
#connection, cursor

Attempting to connect to postgreSQL database...
Connected
postgreSQL db version: ('PostgreSQL 16.2, compiled by Visual C++ build 1937, 64-bit',)


In [9]:
reduced_food.serving_size.max()

256113168.0

In [14]:
reduced_food.available_date.isna().any()

False

In [25]:
branded_food_schema = """
    CREATE TABLE branded_foods (
        fdc_id INTEGER PRIMARY KEY NOT NULL,
        gtin_upc VARCHAR(32),
        serving_size DECIMAL,
        serving_size_unit VARCHAR(4),
        package_weight VARCHAR(32), 
        available_date TIMESTAMP NOT NULL,
        insig_iron BOOLEAN,
        insig_calcium BOOLEAN,
        insig_cholesterol BOOLEAN,
        insig_dietary_fiber BOOLEAN,
        insig_trans_fat BOOLEAN, 
        insig_satured_fat BOOLEAN,
        insig_vitamin_d BOOLEAN,
        insig_potassium BOOLEAN, 
        insig_vitamin_a BOOLEAN,
        insig_vitamin_c BOOLEAN,
        insig_added_sugars BOOLEAN,
        insig_total_sugars BOOLEAN,
        insig_calories_from_fat BOOLEAN, 
        insig_sugars BOOLEAN, 
        insig_fiber BOOLEAN,
        category_id INTEGER,
        brand_owner_id INTEGER,
        brand_name_id INTEGER
    )
"""

In [26]:
cursor.execute(branded_food_schema)

In [24]:
connection.rollback()